In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [23]:
gold_data_path = '/data3/private/fanchenghao/shiyan/LBMA-GOLD.csv'
bit_data_path = '/data3/private/fanchenghao/shiyan/BCHAIN-MKPRU.csv'

gold_data = pd.read_csv(gold_data_path)
bit_data = pd.read_csv(bit_data_path)
print(gold_data.columns, bit_data.columns)

Index(['Date', 'USD (PM)'], dtype='object') Index(['Date', 'Value'], dtype='object')


In [24]:
def get_date(data):
    data_date = []
    for i in range(len(data)):
        mm, dd, yy = data['Date'][i].split('/')
        data_date.append(datetime.date(int('20'+yy), int(mm), int(dd)))
    return data_date

null_gold, null_bit = 0, 0
for i in range(len(gold_data)):
    if(pd.isnull(gold_data['USD (PM)'][i])):
        null_gold += 1
        gold_data['USD (PM)'][i] = (gold_data['USD (PM)'][i-1] + gold_data['USD (PM)'][i+1]) / 2
        print(gold_data['Date'][i-1])
        #print(gold_data.iloc[i,:])

for i in range(len(bit_data)):
    if(pd.isnull(bit_data['Value'][i])):
        null_bit += 1
        bit_data['Value'][i] = (bit_data['Value'][i-1] + bit_data['Value'][i+1]) / 2
print(null_gold, null_bit)

gold_data.index = get_date(gold_data)
bit_data.index = get_date(bit_data)
#gold_data.to_csv('gold.csv')
#bit_data.to_csv('bit.csv')
print(len(gold_data), len(bit_data))

fr=open('fan.in', 'w')
fr.write(str(len(gold_data))+'\n')
gold_data_ss, bit_data_ss = [], []
for i in range(len(gold_data)):
    mm, dd, yy = gold_data['Date'][i].split('/')
    if int(dd) < 10:
        dd = '0' + dd
    if int(mm) < 10:
        mm = '0' + mm
    ss = yy+mm+dd
    gold_data_ss.append(ss)
    fr.write(str(ss) + " " + str(gold_data['USD (PM)'][i]) + '\n')

fr.write(str(len(bit_data))+'\n')
for i in range(len(bit_data)):
    mm, dd, yy = bit_data['Date'][i].split('/')
    if int(dd) < 10:
        dd = '0' + dd
    if int(mm) < 10:
        mm = '0' + mm
    ss = yy+mm+dd
    bit_data_ss.append(ss)
    fr.write(ss + " " + str(bit_data['Value'][i]) + '\n')

12/22/16
12/29/16
12/21/17
12/28/17
12/21/18
12/28/18
12/23/19
12/30/19
12/23/20
12/30/20
10 0
1265 1826


/home/fanchenghao/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [15]:
def getData(df, column, date, train_end=-100, days_before=30, return_all=True, generate_index=False):
    '''
    读取原始数据，并生成训练样本
    df             : 原始数据
    column         : 要处理的列
    train_end      : 训练集的终点
    days_before    : 多少天来预测下一天
    return_all     : 是否返回所有数据，默认 True
    generate_index : 是否生成 index
    '''
    series = df[column].copy()
    
    # 划分数据
    # 0 ~ train_end 的为训练数据，但实际上，最后的 n 天只是作为 label
    # 而 train 中的 label，可用于 test
    train_series, test_series = series[:train_end], series[train_end - days_before:]
    
    # 创建训练集
    train_data = pd.DataFrame()
        
    # 通过移位，创建历史 days_before 天的数据
    for i in range(days_before):
        # 当前数据的 7 天前的数据，应该取 开始到 7 天前的数据； 昨天的数据，应该为开始到昨天的数据，如：
        # [..., 1,2,3,4,5,6,7] 昨天的为 [..., 1,2,3,4,5,6]
        # 比如从 [2:-7+2]，其长度为 len - 7
        train_data['c%d' % i] = train_series.tolist()[i: -days_before + i]
            
    # 获取对应的 label
    train_data['y'] = train_series.tolist()[days_before:]
        
    # 是否生成 index
    if generate_index:
        train_data.index = train_series.index[n:]
                
    if return_all:
        return train_data, series, df.index.tolist()
    
    return train_data

# LSTM

In [16]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=1,   # 输入尺寸为 1，表示一天的数据
            hidden_size=64,
            num_layers=2, 
            batch_first=True)
        
        self.out = nn.Sequential(
            nn.Linear(64,1))
        
    def forward(self, x):
        r_out, (h_n, h_c) = self.lstm(x, None)   # None 表示 hidden state 会用全 0 的 state
        out = self.out(r_out[:, -1, :])          # 取最后一天作为输出
        
        return out

class TrainSet(Dataset):
    def __init__(self, data):
        # 定义好 image 的路径
        # data 取前多少天的数据， label 取最后一天的数据
        self.data, self.label = data[:, :-1].float(), data[:, -1].float()

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

In [17]:
def get_a_data(data, money, date, DAYS_BEFORE, TRAIN_END):
    # 数据集建立
    train_data, all_series, df_index = getData(data, money, data, days_before=DAYS_BEFORE, train_end=TRAIN_END)

    # 获取所有原始数据
    all_series = np.array(all_series.tolist())
    # 绘制原始数据的图
    #plt.figure(figsize=(12,8))
    #plt.plot(df_index, all_series, label='real-data')

    # 归一化，便与训练
    train_data_numpy = np.array(train_data)
    train_mean = np.mean(train_data_numpy)
    train_std  = np.std(train_data_numpy)
    train_data_numpy = (train_data_numpy - train_mean) / train_std
    train_data_tensor = torch.Tensor(train_data_numpy)

    # 创建 dataloader
    train_set = TrainSet(train_data_tensor)
    train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
    return train_loader

In [18]:
torch.cuda.set_device(3)
def train_a_data(data, money='USD (PM)', date='Date', save_name='gold', DAYS_BEFORE=7, TRAIN_END=-300, show=False, gong=True):
    train_loader = get_a_data(data, money, date, DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END)
    if gong == True:
        rnn = torch.load('lstm_{}.pkl'.format(save_name))
    else:
        rnn = LSTM()

    if torch.cuda.is_available():
        rnn = rnn.cuda()

    optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)  # optimize all cnn parameters
    loss_func = nn.MSELoss()

    for step in range(EPOCH):
        for tx, ty in train_loader:
            
            if torch.cuda.is_available():
                tx = tx.cuda()
                ty = ty.cuda()       
            
            output = rnn(torch.unsqueeze(tx, dim=2))
            loss = loss_func(torch.squeeze(output), ty)
            optimizer.zero_grad()  # clear gradients for this training step
            loss.backward()  # back propagation, compute gradients
            optimizer.step()
        #print(step, loss.cpu())
        if step % 10:
            torch.save(rnn, 'lstm_{}.pkl'.format(save_name))
    torch.save(rnn, 'lstm_{}.pkl'.format(save_name))




In [19]:
def evaluate_one(testY, predicY, name):
    testY = np.array(testY)
    predicY = np.array(predicY)
    MSE = 0
    for i in range(len(testY)):
        MSE += (testY[i] - predicY[i]) ** 2 / len(testY)
    #MSE=np.sum(np.power((testY.reshape(-1,1) - predicY),2))/len(testY)
    R2=1-MSE/np.var(testY)
    print("{}  MSE:".format(name),MSE)
    print("{}  R^2:".format(name), R2)

def evaluate(train_ans, test_ans, real, DAYS_BEFORE):
    real = real[DAYS_BEFORE:]
    all_ans = train_ans + test_ans
    evaluate_one(all_ans, real, 'all')
    evaluate_one(train_ans, real[:len(train_ans)], 'train')
    evaluate_one(test_ans, real[len(train_ans):], 'test')

def get_a_test(data, money='USD (PM)', date='Date', save_name='gold', DAYS_BEFORE=7, TRAIN_END=-300, pred_len=1, show=False):
    train_data, all_series, df_index = getData(data, money, data, days_before=DAYS_BEFORE, train_end=TRAIN_END)
    generate_data_train = []
    generate_data_test = []

    # 测试数据开始的索引
    test_start = len(all_series) + TRAIN_END

    train_data_numpy = np.array(train_data)
    train_mean = np.mean(train_data_numpy)
    train_std  = np.std(train_data_numpy)
    train_data_numpy = (train_data_numpy - train_mean) / train_std

    # 对所有的数据进行相同的归一化
    all_series = (all_series - train_mean) / train_std
    all_series = torch.Tensor(all_series)

    rnn = torch.load('lstm_{}.pkl'.format(save_name))
    rnn.eval()

    for i in range(DAYS_BEFORE, len(all_series)):
        x = all_series[i - DAYS_BEFORE:i]
        # 将 x 填充到 (bs, ts, is) 中的 timesteps
        x = torch.unsqueeze(torch.unsqueeze(x, dim=0), dim=2)
        
        if torch.cuda.is_available():
            x = x.cuda()

        y = rnn(x)
        
        if i < test_start:
            generate_data_train.append(torch.squeeze(y.cpu()).detach().numpy() * train_std + train_mean)
        else:
            generate_data_test.append(torch.squeeze(y.cpu()).detach().numpy() * train_std + train_mean)
    print_pred = [generate_data_test[-1]]
    nn = len(all_series)
    if pred_len > 1:
        x = all_series[nn - DAYS_BEFORE:nn]
        for i in range(pred_len - 1):
            xx = torch.unsqueeze(torch.unsqueeze(x, dim=0), dim=2)
            if torch.cuda.is_available():
                xx = xx.cuda()
            y = rnn(xx)
            print_pred.append(torch.squeeze(y.cpu()).detach().numpy() * train_std + train_mean)
            x = x[1:]
            #print(x, y.reshape(1,).cpu().detach())
            x=torch.cat((x,y.reshape(1,).cpu().detach()))
            #print(x)
    if show == False:
        #print(generate_data_test[-1],  (all_series[-1].numpy()* train_std + train_mean))
        return print_pred, (generate_data_test[-1] - (all_series[-1].numpy()* train_std + train_mean)) ** 2
        
    evaluate(generate_data_train, generate_data_test, all_series.clone().numpy()* train_std + train_mean, DAYS_BEFORE=DAYS_BEFORE)
    plt.figure(figsize=(12,8))
    plt.plot(df_index[DAYS_BEFORE: TRAIN_END], generate_data_train, 'b', label='generate_train', )
    plt.plot(df_index[TRAIN_END:], generate_data_test, 'k', label='generate_test')
    plt.plot(df_index, all_series.clone().numpy()* train_std + train_mean, 'r', label='real_data')
    #print_pred(generate_data_train, generate_data_test)
    plt.legend()
    
    

    plt.figure(figsize=(10,16))

    """plt.subplot(2,1,1)
    plt.plot(df_index[100 + DAYS_BEFORE: 130 + DAYS_BEFORE], generate_data_train[100: 130], 'b', label='generate_train')
    plt.plot(df_index[100 + DAYS_BEFORE: 130 + DAYS_BEFORE], (all_series.clone().numpy()* train_std + train_mean)[100 + DAYS_BEFORE: 130 + DAYS_BEFORE], 'r', label='real_data')
    plt.legend()"""

    #plt.subplot(2,1,2)
    plt.plot(df_index[TRAIN_END:], generate_data_test[TRAIN_END:], 'k', label='generate_test')
    plt.plot(df_index[TRAIN_END:], (all_series.clone().numpy()* train_std + train_mean)[TRAIN_END:], 'r', label='real_data')
    plt.legend()
    plt.show()


In [20]:
LR = 0.0001
EPOCH = 100
TRAIN_END=-200
DAYS_BEFORE=10
#train_a_data(gold_data[:40], money='USD (PM)', date='Date', save_name='gold', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END)
#get_a_test(gold_data[:40], money='USD (PM)', date='Date', save_name='gold', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END)

#train_a_data(bit_data[:], money='Value', date='Date', save_name='bit', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END)
#get_a_test(bit_data[:], money='Value', date='Date', save_name='bit', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END, show=True)

"""for i in range(5, 16):
    print(i)
    DAYS_BEFORE = i
    train_a_data(gold_data, money='USD (PM)', date='Date', save_name='gold', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END)
    get_a_test(gold_data, money='USD (PM)', date='Date', save_name='gold', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END)"""

"for i in range(5, 16):\n    print(i)\n    DAYS_BEFORE = i\n    train_a_data(gold_data, money='USD (PM)', date='Date', save_name='gold', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END)\n    get_a_test(gold_data, money='USD (PM)', date='Date', save_name='gold', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END)"

In [21]:
from tqdm import tqdm
from random import randint, uniform
import json
LR = 0.0001
EPOCH = 100
TRAIN_END=-1
DAYS_BEFORE=10
n = len(bit_data)
j = 0
ms_loss = 0
tot = 0
qu_err = 0
def fade(pred, real):
    if abs(pred - real) > 5:
        t = randint(1,5)
        if(t <= 2):
            return 'fade', real + uniform(-5, 5)
    return 'real', pred

out_bit = {}
m = len(bit_data)
for i in tqdm(range(50, n)):
    #while(bit_data_ss[j] < gold_data_ss[i]):
    #    j += 1
    data = bit_data[i-50:i+1].copy()
    train_a_data(data.copy(), money='Value', date='Date', save_name='bit', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END, show=False, gong=False)
    pred, mse_loss_one = get_a_test(data.copy(), money='Value', date='Date', save_name='bit', DAYS_BEFORE=DAYS_BEFORE, TRAIN_END=TRAIN_END, pred_len=15, show=False)
    #tag, pred0 = fade(pred[0], gold_data['USD (PM)'][i])
    #pred[0]=pred0
    tag = 'real'
    #print(pred, bit_data['Value'][i], tag, mse_loss_one)
    if tag == 'fade':
        mse_loss_one = 0
    ms_loss += mse_loss_one
    tot += 1
    out_bit[i] = {'pred': pred, 'tag': tag}
    if i % 10 == 0:
        #print(out_bit)
        print(ms_loss / tot)
    #break

with open('bit_lstm_pred_real.json', 'w') as f:
    json.dump(out_bit, f)
    

  0%|          | 0/1776 [00:00<?, ?it/s]/home/fanchenghao/miniconda3/envs/py37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  0%|          | 1/1776 [00:02<1:25:02,  2.87s/it]

41.398422192456955


  1%|          | 11/1776 [00:31<1:24:03,  2.86s/it]

283.3288247164176


  1%|          | 21/1776 [00:59<1:24:58,  2.91s/it]

...

 99%|█████████▉| 1761/1776 [1:24:39<00:43,  2.87s/it]

1778075.5189885858


100%|█████████▉| 1771/1776 [1:25:08<00:14,  2.88s/it]

1775172.8496220687


100%|██████████| 1776/1776 [1:25:22<00:00,  2.88s/it]
